In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import scipy
import os,sys
import matplotlib.pyplot as plt
import scipy.io as scio
import numpy as np
import tensorflow.keras.optimizers as optimizers
import keras.backend as K
import keras

from keras import Input, regularizers
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneGroupOut,LeavePGroupsOut
from keras.layers import Flatten,Dense,Dropout
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import Input, regularizers
from sklearn.ensemble import RandomForestClassifier

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

print("Part One,Done")


Using TensorFlow backend.


Part One,Done


In [2]:
path=''
data3_Slbp=scio.loadmat(path+'Nblk_Slbp_GLobal_EP')
print(data3_Slbp.keys())

Slbp_data=data3_Slbp.get('Nblk_Slbp_GLobal_EP')
Slbp_label=(data3_Slbp.get('infect_LABEL')).reshape(-1)
Slbp_truelabel=(data3_Slbp.get('trueLABEL')).reshape(-1)
Slbp_rat_label=(data3_Slbp.get('rat_LABEL')).reshape(-1)
print(Slbp_data.shape,Slbp_label.shape,Slbp_truelabel.shape,Slbp_rat_label.shape)

data1=np.loadtxt('001')
data1=data1[0:300]
data2=np.loadtxt('002')
data_resnet=np.concatenate([data1,data2],axis=0)
data=np.concatenate([Slbp_data,data_resnet],axis=1)



print(data1.shape)
print(data2.shape,data_resnet.shape,data.shape)

dict_keys(['__header__', '__version__', '__globals__', 'Nblk_Slbp_GLobal_EP', 'RATTunum', 'infect_LABEL', 'rat_LABEL', 'trueLABEL'])
(600, 7552) (600,) (40,) (600,)
(300, 4)
(300, 4) (600, 4) (600, 7556)


In [3]:
time=0
lpgo = LeavePGroupsOut(n_groups=1)
#clf =SVC(kernel='rbf',class_weight='balanced')
clf = RandomForestClassifier(n_estimators=100)
final_result=np.zeros(40)
mark =0
appear=np.zeros(5)
for(train_index,test_index) in lpgo.split(data,groups=Slbp_rat_label):
    if time>=0 :
        print("第",time,"次留一法")
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = Slbp_label[train_index],Slbp_label[test_index]      
        #print(X_train.shape,y_train.shape)
        clf = RandomForestClassifier(n_estimators=50)
        clf.fit(X_train, y_train)
        score = clf.predict(X_test)
        print(score)
        
        for i in range(15):
            appear[score[i]]+=1
        print(appear)#判为各标签的图谱数量
        
        max_label=np.where(appear==np.max(appear))#max_label[0]为最大值的索引
        for i in range(len(max_label[0])):
            print(max_label[0][i])
            if max_label[0][i]==Slbp_truelabel[time]:        
                mark=1
                print("第",time,"只判别成功")
        if mark==1:
            final_result[time]=1
    mark=0
    time+=1
    appear=np.zeros(5)
    
print(final_result)
print(np.mean(final_result))

第 0 次留一法
[2 2 2 2 2 3 3 2 2 2 3 3 3 3 3]
[0. 0. 8. 7. 0.]
2
第 1 次留一法
[3 3 3 3 1 1 3 1 1 1 2 1 1 4 4]
[0. 7. 1. 5. 2.]
1
第 1 只判别成功
第 2 次留一法
[1 4 4 1 1 1 1 1 4 4 1 4 1 4 4]
[0. 8. 0. 0. 7.]
1
第 2 只判别成功
第 3 次留一法
[4 4 4 1 1 1 1 1 1 1 1 1 1 1 1]
[ 0. 12.  0.  0.  3.]
1
第 3 只判别成功
第 4 次留一法
[4 4 4 1 1 1 1 1 1 1 1 4 4 4 4]
[0. 8. 0. 0. 7.]
1
第 4 只判别成功
第 5 次留一法
[1 1 4 1 1 1 1 1 1 1 1 1 1 1 1]
[ 0. 14.  0.  0.  1.]
1
第 5 只判别成功
第 6 次留一法
[4 4 4 4 4 1 4 4 1 1 1 4 1 1 1]
[0. 7. 0. 0. 8.]
4
第 7 次留一法
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[ 0. 15.  0.  0.  0.]
1
第 7 只判别成功
第 8 次留一法
[4 4 4 1 4 1 1 1 1 1 1 1 1 1 1]
[ 0. 11.  0.  0.  4.]
1
第 8 只判别成功
第 9 次留一法
[4 4 1 1 4 1 4 4 1 1 1 1 1 1 1]
[ 0. 10.  0.  0.  5.]
1
第 9 只判别成功
第 10 次留一法
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[ 0.  0. 15.  0.  0.]
2
第 10 只判别成功
第 11 次留一法
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[ 0.  0. 15.  0.  0.]
2
第 11 只判别成功
第 12 次留一法
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[ 0.  0. 15.  0.  0.]
2
第 12 只判别成功
第 13 次留一法
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[ 0.  0. 15.  0.  0.]


In [ ]:
time=0
lpgo = LeavePGroupsOut(n_groups=1)
#clf =SVC(kernel='rbf',class_weight='balanced')
clf = RandomForestClassifier(n_estimators=200)
final_result=np.zeros(41)
mark =0
for(train_index,test_index) in lpgo.split(data,groups=Slbp_rat_label):
    if time>=0 :
        print("第",time,"次留一法")
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = Slbp_label[train_index],Slbp_label[test_index]      
        #print(X_train.shape,y_train.shape)
        clf = RandomForestClassifier(n_estimators=150)
        clf.fit(X_train, y_train)
        score = clf.predict(X_test)
        print(score)
        
        for i in range(15):
            appear[score[i]]+=1
        print(appear)#判为各标签的图谱数量
        
        max_label=np.where(appear==np.max(appear))#max_label[0]为最大值的索引
        for i in range(len(max_label[0])):
            print(max_label[0][i])
            if max_label[0][i]==Slbp_truelabel[time]:        
                mark=1
                print("第",time,"只判别成功")
        if mark==1:
            final_result[time]=1
    mark=0
    time+=1
    appear=np.zeros(5)
    
print(final_result)
print(np.mean(final_result))

In [ ]:
from sklearn.svm import SVC

clf =SVC(kernel='rbf',class_weight='balanced')
clf.fit(train_x, train_y)
score = clf.score(test_x,test_y)
print(score)

In [ ]:
### Random Forest Classifier    
from sklearn.ensemble import RandomForestClassifier
 
clf = RandomForestClassifier(n_estimators=150)
clf.fit(train_x, train_y)
score = clf.predict(test_x)
max_value=score

index=0
final_result=[]
if (np.mean(np.equal(max_value, 1).astype(int))==0.5):
        compare_result= np.array([1])
else:
        count = np.bincount(max_value)
        common_value = np.argmax(count)
        compare_result = np.equal(common_value, 4).astype(int)
final_result.append(compare_result)
print(score)
print(compare_result)